**Content Produced by UF Signal Processing Society**

**Authors: Raul Valle**

# Intro to GPU Accelerated Scientific Computing

As the name implies, we will learn how to manage CPU (host) and GPU(device) interactions for performing GPU accelerated computing.

### Acknowledgements

First, I'd like to thank NVIDIA for creating CUDA, it's a super easy-to-learn API for using your GPU and accelerating computations. 

I'd like to thank UF and SPS for their interest in this student organization, giving everyone a platform to share their knowledge and resources.

Finally, I'd like to personally thank Dr. Silva, Dr. Principe, Benjamin Colburn, Dr. Wong, Dr. Shea, Matheus Kunzler-Maldaner and Evan Partidas for their continual support in my academic journey.

## Introduction

Scientific Computing is increasingly important as topics such as data analysis, signal processing and machine learning gain relevance in research. 

Specifically, it's important for synthetic tests where evaluation data can be generated independently from other each-other while the computations are as parallelizable as possible.

Another case in which it's important is in any real-time system which is computation heavy and operates in a sequential but parallelizable manner. (We will discuss this more thoroughly soon).

The intent of this workshop is to introduce the user to basic libraries and APIs for GPU accelerating their own scripts, mainly by using the RAPIDS SDK for CuPy and the Numba API for GPU utilization.

## 1. Pre-requisites

There are actually very few pre-requisites for your understanding of using GPUs in your program. Potentially, having a Digital Design and Computer Architecture background will aid in your understanding of how to minimize latencies, however we will not discuss latencies in depth (if you are interested, be on the lookout for the *Hardware Accelerated Scientific Computing* and the *Introduction to Real Time Signal Processing* workshops TBA in the future!).

On the other hand, there are pre-requisites for following this workshop. At the very least, you should have a basic understanding of *Python* and if you don't, it still shouldn't be a major issue as it's almost psuedocode.

To get started, we will need to get your environment set up. We will be using the conda package manager to install the necessary packages. If you don't have conda installed, you can download it from here: https://www.anaconda.com/download/success

Now we need to actually setup the conda environment for the following workshop. Run the following commands (or their OS equivalent) in your terminal.

```
conda env create --name SPS
conda activate SPS
```

Then, we need to install the relevant libraries for this environment.

In [ ]:
%conda install ipykernel numba -c rapidsai -c nvidia -c conda-forge rapids=24.06 python=3.11 cuda-version=12.2

Make sure that your conda environment is selected as your jupyter kernel wherever that selection is made.

Now we are ready to get to work!

## 2. CPU

This is a section on CPU usage, specifically for computing. This section is relatively straight forward for the average programmer, but its purpose is to incentivize the usage of more scalable and efficient methods.

### 2.1. Regular

CPUs or *Central Processing Units* are what your computer uses for basically everything. They run all commands on your computer at a very fast pace (CPU clocks operate between 2 and 5 GHz, and can perform at least 1 instruction per second or *IPS*). 

Assuming the bare minimum, that is, the instructions being run are of SISD type (*Single Input Single Data*), your computer can perform at least 2 Billion instructions per second which is a pretty large amount of operations for such a short period of time. Let's generate an example and time it to put things into perspective.

In [2]:
import time
import numpy as np

tic = time.perf_counter()
x = [i for i in range(100_000)]
toc = time.perf_counter()
print(f"List comprehension: {toc - tic:0.4f} seconds")

tic = time.perf_counter()
y = [2*i+1 for i in x]
toc = time.perf_counter()
print(f"List operation: {toc - tic:0.4f} seconds")

y = [2*i+1 for i in range(1_000)]
tic = time.perf_counter()
z = np.convolve(x, y)
toc = time.perf_counter()
print(f"Convolution: {toc - tic:0.4f} seconds")

List comprehension: 0.0021 seconds
List operation: 0.0148 seconds
Convolution: 0.0470 seconds


While some simple operations perform relatively well, it is clear that more complex compuations (like Convolutions) can take over 10x as long.

Moreover, ponder the situation if you further need to scale these operations to a larger array size, or potentially even having multiple dimensions (as when we work with images or videos).

### 2.2. Vectorization

To address the previous issues (from the CPU), CPU architectures were advanced to support vector-robust methods, or *Vectorized Instructions*. *Vectorized Instructions* are instructions with memory access patterns embedded into the instruction, that is that they are SIMD type (*Single Instruction Multiple Data*).

By instead making use of these kinds of instructions, we can make significant gains in performance - however, there are downsides to this. Python is built on-top of C++, which means that to take advantage of these improvements there need to be strict implications about the data's memory access. The primary down-side is that Python Lists are made to handle objects and be mutable, whereas Vectorized-Access supported lists (which are associated with Numpy) must remain constant-sized and have a fixed data-type.

Let's see the results:

In [6]:
x = np.empty(100_000)
y = np.empty(100_000)
z = np.empty(199_999)

tic = time.perf_counter()
x = np.arange(100_000)
toc = time.perf_counter()
print(f"Array creation: {toc - tic:0.4f} seconds")

tic = time.perf_counter()
y = 2*x+1
toc = time.perf_counter()
print(f"Array operation: {toc - tic:0.4f} seconds")

y = 2*np.arange(1_000)+1
tic = time.perf_counter()
z = np.convolve(x, y)
toc = time.perf_counter()
print(f"Convolution: {toc - tic:0.4f} seconds")

Array creation: 0.0003 seconds
Array operation: 0.0004 seconds
Convolution: 0.0284 seconds


We can see that there are some clear timing benefits of these implementations when directly compared, however by "playing" around with these numbers, these vectorized instructions are not very scalable and actually they fall short the larger the arrays are.

## 3. GPU

This is a section on GPU usage, specifically for computing. This section dives into the most fundamental topics involved in this kind of programming, and displays the scalability of these methods.

* **Note**: The code in this section involving GPU Device utility will often need to be ran twice. The reason for this is that the CPU needs to communicate to the GPU the *context* of the processing it will do. We will mention this again later.

### 3.1 Hardware Architecture

First, some brief insights on the Hardware architecture to give some insights into how your computer system will work.

We mentioned that the CPU is the *Central Processing Unit* and this naming convention will be relevant as shown later. However, when it was introduced previously, some might be under the notion that the CPU is 1 unified piece of hardware, it is not.

Actually, modern CPUs have 2 main configurations:

*Single-Die, Multi-Core CPU* (SDMC CPU) or *Multi-Die, Multi-Core CPU* (MDMC CPU)

There are tradeoffs with each design, most notably: 

SDMC CPUs: **less** threads, **faster** clock speed

MDMC CPUs: **more** threads, **slower** clock speed

Within each die are multiple cores, which each contain multiple threads. Each *core* can run its own program, however certain kinds of programs can take advantage of the individual *threads* in a core to improve computational efficiency. We won't get into the specifics of threading yet, we will reserve that for a future workshop -

However, this should give some insight into the utility of a GPU. Originally named the *Graphics Processing Unit*, a GPU is really just a CPU optimized for vectorized operations.

In this case, if we were to continue the comparisons above:

MDMC CPUs: **more** threads, **slower** clock speed, **flexible** instruction set

GPUs: **even more** threads, **even slower** clock speed, **limited** instruction set

Hopefully, you have an idea about the structure for the system in your computer as a result of this.

### 3.2. System Structure

Recall that previously we called the CPU the *host* and the GPU the *device*, these are the terminology used to refer to these two components in our computers.

The CPU is considered the *host* because it queues instructions for the corresponding *device* to follow. The reason is simple, since the CPU operates at much higher clock speeds, it can queue instructions for the *device* to perform at a fast enough rate. 

The GPU is considered the *device* because it takes instructions from the *host* and distributes them among its multiple threads.

However, the structure of the GPU threads are further organized in comparison to CPUs. Typically, threads are organized into *warps* which are clusters of 32 threads that execute simultaneously - with this organization, it is possible to hide latencies due to data transfers or computations.

While the organization of threads into warps is meaningful in the context of hiding latencies, it is usually more useful to think about the organization of the threads in the context of your program.

### 3.3. GPU Memory

Now that we know the structure of the system, it should follow that there is a means to access data on the *device(s)* 

**Notes**: The largest latencies in any existing system are memory transfers. Therefore in general, we would like to avoid data transfers whenever possible. They are inevitable in **Real-Time Systems** where data is always handled *"globally"*, however the existence of *warps* help mitigate the latency introduced by these data transfers.

A very simple example:

In [3]:
import cupy as cp

tic = time.perf_counter()
x_d = cp.asarray(x)
toc = time.perf_counter()
print(f"Array creation: {toc - tic:0.4f} seconds")

tic = time.perf_counter()
y_d = 2*x_d+1
toc = time.perf_counter()
print(f"Array operation: {toc - tic:0.4f} seconds")

y_d = 2*cp.arange(1_000)+1
tic = time.perf_counter()
z_d = cp.convolve(x_d, y_d)
toc = time.perf_counter()
print(f"Convolution: {toc - tic:0.4f} seconds")

Array creation: 0.1471 seconds
Array operation: 0.0133 seconds
Convolution: 0.0005 seconds


From this example, it may seem as though the GPU computations are not justified, they actually consume a lot of time in comparison to the CPU operations. Let's now compare a larger scale operation:

In [5]:
tic = time.perf_counter()
x = np.arange(10_000_000)
toc = time.perf_counter()
print(f"Numpy Array creation: {toc - tic:0.4f} seconds")

tic = time.perf_counter()
y = 2*x+1
toc = time.perf_counter()
print(f"Numpy Array operation: {toc - tic:0.4f} seconds")

y = 2*np.arange(100_000)+1
tic = time.perf_counter()
z = np.convolve(x, y)
toc = time.perf_counter()
print(f"Numpy Convolution: {toc - tic:0.4f} seconds")

tic = time.perf_counter()
x_d = cp.asarray(x)
toc = time.perf_counter()
print(f"Cupy Array creation: {toc - tic:0.4f} seconds")

tic = time.perf_counter()
y_d = cp.asarray(y)
toc = time.perf_counter()
print(f"Cupy Array operation: {toc - tic:0.4f} seconds")

y_d = 2*cp.arange(100_000)+1
tic = time.perf_counter()
z_d = cp.convolve(x_d, y_d)
toc = time.perf_counter()
print(f"Cupy Convolution: {toc - tic:0.4f} seconds")

Numpy Array creation: 0.0100 seconds
Numpy Array operation: 0.0175 seconds
Numpy Convolution: 263.1087 seconds
Cupy Array creation: 0.0138 seconds
Cupy Array operation: 0.0002 seconds
Cupy Convolution: 0.0035 seconds


Data transfer clearly introduces a large latency in the computation, but the reduction in complex computing is *massive*, over 300x more efficient (based on this example). Clearly, if the main latency is data transfers, then complex computations that are relevant to Machine Learning and Signal Processing can be significantly reduced by using GPUs. 

Before moving onto the most important part of using GPUs let's show how to transfer data back to the CPU.

In [7]:
tic = time.perf_counter()
x_host = cp.asnumpy(x_d)
toc = time.perf_counter()
print(f"Array transfer (x): {toc - tic:0.4f} seconds")

tic = time.perf_counter()
y_host = cp.asnumpy(y_d)
toc = time.perf_counter()
print(f"Array transfer (y): {toc - tic:0.4f} seconds")

tic = time.perf_counter()
z_host = cp.asnumpy(z_d)
toc = time.perf_counter()
print(f"Array transfer (z): {toc - tic:0.4f} seconds")

Array transfer (x): 0.0258 seconds
Array transfer (y): 0.0003 seconds
Array transfer (z): 0.0166 seconds


### 3.4. GPU Kernels

Now that we have seen how to transfer data between the *host* and the *device*, we will dive into the main function of GPU Accelerated Computing.

In research, there are many times when you will need to implement your own new method. This new method, assuming that it is associated with **Signal Processing** or **Machine Learning**, will often operate on *vectors*, *matrices*, or *tensors* of all shapes and sizes, meaning that there can be a pattern-like memory access and possibility for Parallelization.

Here we will show an example of a kernel and express the key considerations when writing one.

In [10]:
from numba import cuda
import math

@cuda.jit
def randomFeatureMap(x, y, z, N, M, K):
    thidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    thidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y
    thidz = cuda.threadIdx.z + cuda.blockIdx.z * cuda.blockDim.z
    if thidx < N and thidy < M and thidz < K:
        z[thidx, thidy, thidz] = math.exp(x[thidx]/20) * math.log(y[thidy])
    cuda.syncthreads()

Above we built a fully functional kernel which generates a random FeatureMap, which is common in any machine learning or signal processing process.

Clearly, x and y are the 2 input vectors, and z is the 1 output tensor.

N, M and K are inputs that specify the shape of the corresponding input vector and the output tensor. It is possible to simply extract these values by using numba's built-in functions, but it is common practice the values in as an argument.

In a kernel, thidx, thidy and thidz are used to access input objects in a patterned way. It is not necessary to use all 3 indeces, you can use 2 or 1 depending on the program you are writing. 

Under extremely careful construction, it is likely that the if statement to check that the indeces are being correctly bounded are not necessary, however it is standard practice to code this restriction.

You may have noticed that each threadid is expressed as a sum of the threadIdx and some *block* information. The threads are the corresponding threads on the GPU which we have already discussed, however the blocks are clusters of threads, expressing how to "sparsify" the operations for the corresponding output. For very large matrices, it is clear that 1024 threads (the typical amount of threads in a GPU) will not suffice, which is why we need to *blockify* the operations for the output.

If you recall, *warps* are also clusters of threads. The main distinction between blocks and warps is that blocks simply express the total amount of resources that will be necessary to perform a kernel, whereas warps are the physical clusters of 32 threads to be ran immediately and independently from other warps.

Finally ```cuda.syncthreads()``` is a function which will force the GPU to wait for all the threads to finish computing before moving on. This is usually only necessary for more complex kernels with sequential operations.

### 3.5. GPU System

Now, we have discussed all the main tools necessary to create a system that uses a GPU. Let's see how we would use one in practice.

In [21]:
#We are setting the seed to make the results reproducible for everyone
np.random.seed(42)
#We are creating random data, consider that in a real time system we would just have the data,
#but for synthetic data we would have to generate it here

#First let's set the data type and dimensions of the data
N = 2**16
M = 32
K = 16
x = np.empty((N), dtype=np.float32)
y = np.empty((M), dtype=np.float32)

#Now we fill the data with random values
x = np.random.rand(N)*16
y = np.random.rand(M)*16

#Now, we need to allocate memory in the GPU
tic1 = time.perf_counter()
x_d = cuda.to_device(x)
y_d = cuda.to_device(y)
z_d = cuda.device_array((N,M,K), dtype=np.float32)

#Now we create the output array
tic2 = time.perf_counter()
randomFeatureMap[(N//2, 1, 1), (2, M, K)](x_d, y_d, z_d, N, M, K)
cuda.synchronize()

#Now we transfer the data back to the host
tic3 = time.perf_counter()
z_gpu = z_d.copy_to_host()
tic4 = time.perf_counter()

#We can optionally print the results
#print(z)

#Instead, We'd like to emphasize that the computing time is low and accurate
z_temp = np.empty((N,M), dtype=np.float32)
z_cpu = np.empty((N,M,K), dtype=np.float32)
tic5 = time.perf_counter()
for i in range(N):
    for j in range(M):
        z_temp[i,j] = math.exp(x[i]/20) * math.log(y[j])
z_cpu = np.repeat(z_temp[:,:,np.newaxis], K, axis=2)
tic6 = time.perf_counter()

#Let's compare the results
print(f"GPU time: {tic4-tic1:0.4f} seconds")
print(f"CPU time: {tic6-tic5:0.4f} seconds")
#print(f"Transfer latency: {(tic2-tic1)+(tic4-tic3):0.4f} seconds")
print(f"Speedup: {(tic6-tic5)/(tic4-tic1):0.4f} times")

print('GPU == CPU:', np.allclose(z_gpu, z_cpu))

GPU time: 0.0311 seconds
CPU time: 0.4961 seconds
Transfer latency: 0.0310 seconds
Speedup: 15.9352 times
GPU == CPU: True


We just observed (depending on your machine) around or over 10x speedup in computing, and this is for a singular computation. This is clearly very good for scaling computing.

While the results are great and there are clear implications about this speedup, let's dive into the *kernel configuration*.

Typically function calls are made where you just call 

```function(...)``` 

However, kernels are called 

```kernel[(grid_dims),(block_dims)]```

Again returning to the concept of structure, *threads* are organized into *blocks*, and *blocks* are organized into a *grid*.

As mentioned previously, there are many occasions where a block does not iterate over the entire object, which is why a *grid* is necessary to express how to iterate through the object, in terms of the blocks.

Finally, the ```cuda.synchronize()``` function is necessary to ensure that kernels that depend on previous results do not begin until the necessary results have been generated.

## 4. Minimizing Latency

This section is mostly for completeness, but so we will not fully dive into this topic until the sequel workshop, *Hardware Accelerated Scientific Computing* (stay tuned). However we will mention key topics for consideration.

We've already seen an example of a *full system* that makes use of the GPU device, however we have only scratched the surface when it comes to optimizing the GPU's performance. 

It is possible that an implementation like in our example is the most optimal one, especially in cases where *a lot* of data is handled simultaneously. This mostly depends on your GPU's potential which you can check by running ```nvidia-smi``` in your terminal, however typically this is not the case.

There are ways to quantify the efficiency of your program, namely the *compute to global memory access ratio* (CGMA) which is defined as floating-point calculations : global memory access. 

Some intuition for this: High-End GPUs today have *memory-bandwidth* supporting between 500 and 1000 GB/s. On the other hand, they support between 600 and 1300 TFLOPS, that is Terra *Floating Point Operations*. This implies that **optimal** *CGMA* ratios should hang around 1000. 

There are a few ways to hide latency, one which was already addressed (refer to the warps details above), and the other is to build programs with strong consideration for memory transfers/access. A memory transfer from the CPU to the GPU is inevitable, however by making use of the GPU's local resources composed of *registers* and *shared-memory*, it is possible to further reduce memory latency.

We won't dive into access patterns yet, but let's define different memory types. *Device Global Memory* is the GPU's accomadated memory, it has the most amount of memory, implemented using DDR4, it is shared amongst all threads and it is the slowest of the 3. *Device Shared Memory* is the GPU's local memory which is shared amongst threads in a block - this means that as its name implies, the memory is shared amongst these threads, and it is faster than *Device Global Memory*. *Device Local Memory* is also the GPU's local memory which is private amongst the threads in a block, it can only be accessed by its own thread and is the fastest of the memory accesses.

In creating kernels, these are important considerations, however libraries like CuPy (in the next section), take care of most of this for us already.

## 5. NumPy (CPU) and CuPy (GPU)

While it can be a fun (and humbling) learning experience to implement kernels in Numba, we'd like in general to avoid making them from scratch because super corporations like NVIDIA have already implemented most Linear Algebra, Statistics or Machine Learning tools that you would ever need.

My intent is to just introduce some tools/functions that will accelerate your workflow.


```cp.as_array(x)``` sends a numpy array to the device

```cp.to_device(x_d)``` sends a device array to the host

```cp.empty((N,M,K), datatype=cp.float32)``` allocates a device array

```cp.zeros((N,M,K), datatype=cp.float32)``` instantiates a device array with 0s

```cp.repeat(X_d, K, axis=a)``` repeats a device array K times along axis a

```X_d@W_d``` performs a matrix multiplication

```X_d(*)W_d``` performs an element-wise operation (adding,subtracting, etc.)

CuPy and NumPy have very similar workflows, so actually most functions will work nearly identically as long as you specify to use NumPy instead:

```np.empty((N,M,K), datatype=np.float32)``` allocates a host array

```np.zeros((N,M,K), datatype=np.float32)``` instantiates a host array with 0s

```np.repeat(X, K, axis=a)``` repeats a host array K times along axis a

```X@W``` performs a matrix multiplication

```X(*)W``` performs an element-wise operation (adding,subtracting, etc.)

## 6. Conclusion

GPU computing is important because it enables the rapid processing of large-scale computations and data parallelism, significantly accelerating tasks like machine learning, scientific simulations, and image rendering. This enhances performance and efficiency in various fields, including AI, research, and gaming.

We covered topics including: memory management, kernel development and configuration, and workflow acceleration.